In [1]:
# If you run this notebook on Google Colaboratory, uncomment the below to install automl_alex.
#!pip install --quiet automl_alex

In [1]:
import automl_alex
import sklearn
import time
from automl_alex import AutoML, AutoMLClassifier
print(automl_alex.__version__)

0.07.17


In [2]:
RANDOM_SEED = 42

# Classifier

## Data

In [3]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
dataset = fetch_openml(name='credit-g', version=1, as_frame=True)
dataset.target = dataset.target.astype('category').cat.codes
dataset.data.head(5)

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,4.0,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes
1,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,female div/dep/mar,none,2.0,real estate,22.0,none,own,1.0,skilled,1.0,none,yes
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,male single,none,3.0,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,yes
3,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,male single,guarantor,4.0,life insurance,45.0,none,for free,1.0,skilled,2.0,none,yes
4,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,male single,none,4.0,no known property,53.0,none,for free,2.0,skilled,2.0,none,yes


In [4]:
X_train, X_test, y_train, y_test = train_test_split(dataset.data, 
                                                    dataset.target,
                                                    test_size=0.2, 
                                                    random_state=RANDOM_SEED,)
X_train.shape, X_test.shape

((800, 20), (200, 20))

## Model

In [5]:
model = AutoML(X_train, y_train, X_test, type_of_estimator='classifier', random_state=RANDOM_SEED)

# or Simply
model = AutoMLClassifier(X_train, y_train, X_test, random_state=RANDOM_SEED, verbose=1)

Source X_train shape:  (800, 20) | X_test shape:  (200, 20)
##################################################
Auto detect cat features:  13
> Start preprocessing Data
> Generate cat encodet features
 +  55  Features from  OneHotEncoder
 +  44  Features from  HelmertEncoder
 +  54  Features from  HashingEncoder
 +  16  Features from  FrequencyEncoder
> Clean Nans in num features
> Generate interaction Num Features
 +  24  Interaction Features
##################################################
> Total Generated Features:  185
##################################################
New X_train shape:  (800, 201) | X_test shape:  (200, 201)


In [6]:
%%time
predict_test, predict_train = model.fit_predict(timeout=1000, verbose=2)


 Opt StackingModels
One iteration takes ~ 0.8 sec
> Start Auto calibration parameters
> Start optimization with the parameters:
CV_Folds =  10
Score_CV_Folds =  3
Feature_Selection =  True
Opt_lvl =  2
Cold_start =  59.0
Early_stoping =  100
Metric =  roc_auc_score
Direction =  maximize
##################################################
Default model OptScore = 0.5939
Optimize: : 209it [11:46,  3.38s/it,  | Model: CatBoost | OptScore: 0.7822 | Best roc_auc_score: 0.8097 +- 0.027544]

 Predict from StackingModels
 10%|█         | 1/10 [01:12<10:48, 72.01s/it]
 Mean Score roc_auc_score on 30 Folds: 0.7928 std: 0.059757
 20%|██        | 2/10 [01:22<07:07, 53.43s/it]
 Mean Score roc_auc_score on 30 Folds: 0.7943 std: 0.05901
 30%|███       | 3/10 [01:23<04:24, 37.82s/it]
 Mean Score roc_auc_score on 30 Folds: 0.7726 std: 0.0615
 40%|████      | 4/10 [01:24<02:41, 26.89s/it]
 Mean Score roc_auc_score on 30 Folds: 0.7715 std: 0.062109
 50%|█████     | 5/10 [01:26<01:35, 19.19s/it]
 Mean Sco

In [7]:
predict_test[:5]

array([0.54027738, 0.57470047, 0.76622332, 0.452619  , 0.14477764])

In [8]:
print('Test AUC: ', round(sklearn.metrics.roc_auc_score(y_test, predict_test),4))

Test AUC:  0.8216


In [10]:
model.stack_models_cfgs

,score_opt,model_score,score_std,model_name,model_param,wrapper_params,cat_encoders,columns,cv_folds
179,0.7822,0.8097,0.027544,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, num_dependents, OneHotEncoder_job_3...",10
189,0.7720,0.8153,0.043300,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, num_dependents, OneHotEncoder_job_3...",10
73,0.7529,0.7735,0.020599,LinearModel,"{'verbose': 0, 'fit_intercept': True, 'C': 61....",{},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, num_dependents, OneHotEncoder_job_3...",10
68,0.7514,0.7746,0.023156,LinearModel,"{'verbose': 0, 'fit_intercept': True, 'C': 74....",{},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, num_dependents, OneHotEncoder_job_3...",10
119,0.7505,0.7741,0.023554,LinearModel,"{'verbose': 0, 'fit_intercept': True, 'C': 83....",{},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, num_dependents, OneHotEncoder_job_3...",10
165,0.7503,0.7854,0.035122,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, num_dependents, OneHotEncoder_job_3...",10
198,0.7482,0.7908,0.042579,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, num_dependents, OneHotEncoder_job_3...",10
71,0.7473,0.7716,0.024275,LinearModel,"{'verbose': 0, 'fit_intercept': True, 'C': 68....",{},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, num_dependents, OneHotEncoder_job_3...",10
60,0.7468,0.7711,0.024293,LinearModel,"{'verbose': 0, 'fit_intercept': True, 'C': 98....",{},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, num_dependents, OneHotEncoder_job_3...",10
72,0.7468,0.7744,0.027643,LinearModel,"{'verbose': 0, 'fit_intercept': True, 'C': 69....",{},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, num_dependents, OneHotEncoder_job_3...",10
